In [1]:
train = pd.read_json('./raw_data/train.json')
# val = pd.read_json('./raw_data/val.json')

In [110]:
song_meta = pd.read_json('./raw_data/song_meta.json')

In [ ]:
raw_df = pd.read_csv('./df_for_MLC.csv')

## 노래별로 어떤 tag가 붙어 있는지

In [3]:
ls = []
for i in train[['tags','songs']].index:
    tags = train.get_value(i,'tags')
    songs = train.get_value(i,'songs')
    for song in songs:
        ls.append({'id':song,'tags': tags})
song_tag = pd.DataFrame(ls)
song_tag_df = song_tag.groupby('id').agg(sum).reset_index()
song_tag_df

,id,tags
0,0,"[비오는날, 드라이브, 비오는날, 추억, 회상]"
1,3,"[1, 감성, 듣기좋은, 드라이브, Pop, 여행, 기분전환, 팝, 신나는, 클럽,..."
2,4,"[뉴에이지, 뉴에이지]"
3,5,"[비오는날, 재즈, 락, 발라드, 휴식, 리메이크, 재즈, 힐링송, 힐링, 기분전환..."
4,6,"[클래식, 산책, 조깅]"
...,...,...
615137,707984,[월드뮤직]
615138,707985,"[Pop, 해외차트, 시대별차트, 신디로퍼, Pop, 1980, 라이벌, 록]"
615139,707986,"[힙합, 랩, 발라드, 감성, 리메이크곡, 카페, 잔잔한, 추억, 위로, 힐링, 사..."
615140,707987,"[집중, 노동요, 새벽, 오후, 자장가, 공부]"


# train playlist에 언급된 unique한 노래들 추출

In [4]:
df_songs = train['songs'].apply(lambda x : [str(i) for i in x])
df_songs = train['songs'].reset_index(drop=True)
unique_song_list = np.concatenate(df_songs)
unique_song_list = list(set(unique_song_list))
unique_song_list = [int(i) for i in unique_song_list]
unique_song_list[:5]

[0, 3, 4, 5, 6]

In [5]:
df = raw_df[raw_df['id'].isin(unique_song_list)]

In [6]:
df

,id,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,...,GN2900,GN2901,GN2902,GN2903,GN9000,issue_year,issue_month,like_cnt,population,tag_score
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2014,5,29.0,2.0,24176.0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2015,10,1686.0,8.0,79205.0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2011,8,39.0,2.0,3493.0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2010,6,391.0,5.0,59069.0
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2017,2,24.0,1.0,4982.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707984,707984,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1999,12,2.0,1.0,243.0
707985,707985,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1986,0,11.0,4.0,4323.0
707986,707986,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,2016,1,1123.0,11.0,162809.0
707987,707987,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2013,12,4.0,1.0,12775.0


## 연도를 one-hot-encoding 진행

In [7]:
df[['issue_year','issue_month']] = df[['issue_year','issue_month']].apply(lambda x : [str(i) for i in x])

In [8]:
final_df_year_month_dummies = pd.get_dummies(df[['issue_year','issue_month']])

In [9]:
final_df_year_month_dummies.reset_index(inplace=True)
final_df_year_month_dummies.rename(columns={'index':'id'},inplace=True)

In [10]:
final_df_one_hot = df[df.columns.difference(['issue_year','issue_month'])].merge(final_df_year_month_dummies, on='id')

In [11]:
final_df_one_hot

,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,GN0203,...,issue_month_12,issue_month_2,issue_month_3,issue_month_4,issue_month_5,issue_month_51,issue_month_6,issue_month_7,issue_month_8,issue_month_9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615137,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
615138,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
615139,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
615140,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## id만 발라내기

In [12]:
final_df_no_id = final_df_one_hot[final_df_one_hot.columns.difference(['id'])]

In [13]:
final_df_no_id[['population','like_cnt','tag_score']]

,population,like_cnt,tag_score
0,2.0,29.0,24176.0
1,8.0,1686.0,79205.0
2,2.0,39.0,3493.0
3,5.0,391.0,59069.0
4,1.0,24.0,4982.0
...,...,...,...
615137,1.0,2.0,243.0
615138,4.0,11.0,4323.0
615139,11.0,1123.0,162809.0
615140,1.0,4.0,12775.0


## 정규화

In [14]:
from sklearn.preprocessing import minmax_scale

In [15]:
MinMax_scaled = minmax_scale(final_df_no_id, axis=0, copy=True, )

In [16]:
df_for_KNN = pd.DataFrame(MinMax_scaled, columns = final_df_no_id.columns)

In [17]:
df_for_KNN['id'] = final_df_one_hot['id']

In [18]:
df_for_KNN

,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,GN0203,...,issue_year_2016,issue_year_2017,issue_year_2018,issue_year_2019,issue_year_2020,issue_year_2022,like_cnt,population,tag_score,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000089,0.00046,0.065164,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.005157,0.00322,0.213495,3
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000119,0.00046,0.009413,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.001196,0.00184,0.159218,5
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.000073,0.00000,0.013426,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000006,0.00000,0.000652,707984
615138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000034,0.00138,0.011650,707985
615139,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.003435,0.00460,0.438850,707986
615140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000012,0.00000,0.034432,707987


In [19]:
song_tag_df

,id,tags
0,0,"[비오는날, 드라이브, 비오는날, 추억, 회상]"
1,3,"[1, 감성, 듣기좋은, 드라이브, Pop, 여행, 기분전환, 팝, 신나는, 클럽,..."
2,4,"[뉴에이지, 뉴에이지]"
3,5,"[비오는날, 재즈, 락, 발라드, 휴식, 리메이크, 재즈, 힐링송, 힐링, 기분전환..."
4,6,"[클래식, 산책, 조깅]"
...,...,...
615137,707984,[월드뮤직]
615138,707985,"[Pop, 해외차트, 시대별차트, 신디로퍼, Pop, 1980, 라이벌, 록]"
615139,707986,"[힙합, 랩, 발라드, 감성, 리메이크곡, 카페, 잔잔한, 추억, 위로, 힐링, 사..."
615140,707987,"[집중, 노동요, 새벽, 오후, 자장가, 공부]"


## 분석에 필요한 데이터 남기고

In [29]:
df = df_for_KNN.merge(song_tag_df,on='id')

In [32]:
# 메모리 위해 변수 삭제
del raw_df

## 10000개만 뽑아서 찾아보기

In [215]:
df_10000 = df.sample(10000)

In [220]:
df_10000 = df_10000.set_index(['id'])

In [231]:
y = list(df_10000.tags)

## song- tag one-hot vector 만들기

In [311]:
from sklearn.preprocessing import MultiLabelBinarizer
Y_mlb = MultiLabelBinarizer(sparse_output=True)
Y_true = Y_mlb.fit_transform(y)

In [312]:
kwd_classes = Y_mlb.classes_ # binarizer를 저장해두고, 이후에 classes_ 를 읽어서, 각 칼럼이 무슨 값인지를 확인해야 함.

In [313]:
X = df_10000[df_10000.columns.difference(['tags'])]

In [314]:
X

,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,GN0203,...,issue_year_2015,issue_year_2016,issue_year_2017,issue_year_2018,issue_year_2019,issue_year_2020,issue_year_2022,like_cnt,population,tag_score
id,,,,,,,,,,,,,,,,,,,,,
497989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000046,0.00230,0.116772
58363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000067,0.00000,0.026815
603344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000028,0.00000,0.017316
519851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000006,0.00000,0.051732
539226,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010008,0.00322,0.134980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000049,0.00092,0.145371
560342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010225,0.00874,0.332407
534474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000009,0.00000,0.016825


In [315]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, Y_true)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(7500, 350)
(7500, 17465)
(2500, 350)
(2500, 17465)


## MLPClassifier를 튜닝해야 합니다
- Multi-Output Classification가 저희가 만들어 내야할 모델입니다
- tensorflow에서 구현하는 방향으로 가면 디테일하게 튜닝해볼수 있을것 같습니다.
- 예를들어 지금 이모델은 0,1만 뱉어내는 모델이기 때문에, 최종적으로 sigmoid 함수로 0~1사이의 값을 뱉어내개 해야 합니다.
- 지금 MLPClassifier에서는 그방법을 모르겠고, Tensorflow에서 그 기능을 구현하는 방향으로 진행해야 합니다.

In [ ]:
## 모델링

In [316]:
mlp_multilabel = MLPClassifier(hidden_layer_sizes=(300,100), max_iter=100, 
                               random_state=42).fit(X_train, y_train)

In [317]:
Y_pred = mlp_multilabel.predict(X_test)
Y_pred = Y_pred.toarray()
y_test = y_test.toarray()
np.sum(np.sum(y_test.astype(int) & Y_pred, axis=1) > 0)/y_test.shape[0]

0.0

In [ ]:
mlp_multilabel = MLPClassifier(hidden_layer_sizes=(300,100), max_iter=100, 
                               random_state=42).fit(X_train, y_train)

In [329]:
## 대략적인 accuracy

In [243]:
Y_pred = mlp_multilabel.predict(X_test)
Y_pred = Y_pred.toarray()
y_test = y_test.toarray()
np.sum(np.sum(y_test.astype(int) & Y_pred, axis=1) > 0)/y_test.shape[0]

0.49

In [246]:
X_test

,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,GN0203,...,issue_year_2015,issue_year_2016,issue_year_2017,issue_year_2018,issue_year_2019,issue_year_2020,issue_year_2022,like_cnt,population,tag_score
id,,,,,,,,,,,,,,,,,,,,,
43349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000177,0.000000,0.000073
130292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000006,0.000000,0.056713
251554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000049,0.000000,0.039543
72971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001132,0.000920,0.086132
445599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000288,0.000460,0.026505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000040,0.000000,0.001132
376925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.009461
420697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000003,0.000000,0.016944


## 결과 확인을 위한 작업

In [282]:
result_df = pd.DataFrame(Y_pred, columns=kwd_classes)
result_df = result_df.merge(pd.DataFrame(X_test.index),on=result_df.index)
result_df = result_df.drop('key_0',axis=1).set_index(['id'])

In [322]:
result_df.index

Int64Index([ 43349, 130292, 251554,  72971, 445599, 331223, 583128, 522005,
            511103, 506340,
            ...
            162767, 448050, 152268, 653515, 545933, 674808, 376925, 420697,
            450117, 701975],
           dtype='int64', name='id', length=2500)

## 결과 학인 함수 설정

In [334]:
def get_tags(ls):
    result_ls = []
    for i in ls:
        result_ls.append(result_df.loc[i][result_df.loc[i] > 0])
    return result_ls

In [335]:
pd.DataFrame(get_tags(result_df.index))

,클래식,랩,힙합,재즈,락,기분전환,감성,뉴에이지,잔잔한,카페,...,어두운,울적,이불속에서,인디추천,잠이오지않는,캠핑,피쉬슈즈,오월천,일본노래,샤이니
43349,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130292,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450117,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 보다 싶이 1값을 가지고 있는게 이 모델이 추천해준 태그들입니다. 
- 마찬가지로 태그의 요소들로 인해 뽑아내는 방법을 강구해야 합니다.

In [321]:
song_meta[song_meta['id'] == 450117]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
450117,"[GN0601, GN0606]",20141024,누군가로부터,2287224,[772043],"만남, 이별, 추억 그리고 그리움",[GN0600],[레터 플로우],450117


In [320]:
df_10000[df_10000.index == 450117]

,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,GN0203,...,issue_year_2016,issue_year_2017,issue_year_2018,issue_year_2019,issue_year_2020,issue_year_2022,like_cnt,population,tag_score,tags
id,,,,,,,,,,,,,,,,,,,,,
450117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.013684,0.022999,0.527572,"[잔잔한, 밤, 새벽, 잠, 걱정, 차분한, 생각, 나, 잠들기전, 고민, 편안한,..."


In [319]:
result_df.loc[450117][result_df.loc[450117] > 0]

가을      1
감성      1
겨울      1
기분전환    1
까페      1
드라이브    1
락       1
매장음악    1
발라드     1
밤       1
봄       1
분위기     1
비오는날    1
사랑      1
산책      1
새벽      1
설렘      1
스트레스    1
슬픔      1
신나는     1
여름      1
여행      1
운동      1
이별      1
인디      1
잔잔한     1
추억      1
카페      1
팝       1
회상      1
휴식      1
힐링      1
힙합      1
Name: 450117, dtype: int64

# 결론
- 앞서 말씀드렸듯이, 일단 모델이 돌아가서 예측은 가능하나 역시 0,1인 값만 가진 벡터를 반환하기 때문에, 확률값으로 0~1사이 값을 반환하도록 마지막 output layer 단계에서 sigmoid 함수를 사용해야 할듯합니다.
- 이런 방법론으로 저희가 배웠던 tensorflow의 모델링 방법을 활용해야 할듯 합니다.

# tf.keras 
- 현재 시도중 입니다.

In [192]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Dense(units=1000, input_shape=[350]),
    layers.Dense(y_test.shape[1], activation='sigmoid')
])

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 1000)              351000    
_________________________________________________________________
dense_24 (Dense)             (None, 5168)              5173168   
Total params: 5,524,168
Trainable params: 5,524,168
Non-trainable params: 0
_________________________________________________________________


In [194]:
model.compile(optimizer='adam',
             metrics=['accuracy'])

In [195]:
hist = model.fit(X_train,y_train,
                epochs=5, verbose=1,
                validation_data= (X_test, y_test))

Epoch 1/5


ValueError: in user code:

    /Users/yoohowon/opt/anaconda3/envs/fastcampus/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Users/yoohowon/opt/anaconda3/envs/fastcampus/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/yoohowon/opt/anaconda3/envs/fastcampus/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/yoohowon/opt/anaconda3/envs/fastcampus/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/yoohowon/opt/anaconda3/envs/fastcampus/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:541 train_step  **
        self.trainable

In [157]:
df_10000 = df.sample(1000)

In [158]:
y = list(df_10000.tags)

In [159]:
from sklearn.preprocessing import MultiLabelBinarizer
Y_mlb = MultiLabelBinarizer(sparse_output=False)
Y_true = Y_mlb.fit_transform(y)

In [160]:
kwd_classes = Y_mlb.classes_ # binarizer를 저장해두고, 이후에 classes_ 를 읽어서, 각 칼럼이 무슨 값인지를 확인해야 함.

In [161]:
kwd_classes

array(['00년', '00년대', '015B', ..., '힛뎀포크', '힛뎀폭', '힛뎀폭스'], dtype=object)

In [162]:
Y_true

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [164]:
X = np.array(df_10000[df_10000.columns.difference(['id','tags'])])

In [165]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, Y_true)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(750, 350)
(750, 5168)
(250, 350)
(250, 5168)
